<a href="https://colab.research.google.com/github/HaJunYoo/Pyspark-tutorial/blob/main/movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### module install

In [8]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 
!pip install -q findspark

import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

MAX_MEMORY = "6g"

conf = SparkConf()
conf.set("spark.app.name", "movie-recommendation")
conf.set("spark.master", "local[*]")
conf.set("spark.executor.memory", MAX_MEMORY)
conf.set("spark.driver.memory", MAX_MEMORY)

# Singleton pattern
spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

### movie lens data get

In [11]:
#importing the movie lens dataset directly to colab
!wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2023-03-01 15:38:13--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip.1’

ml-25m.zip.1        100%[===================>] 249.84M  29.2MB/s    in 9.3s    

2023-03-01 15:38:23 (26.8 MB/s) - ‘ml-25m.zip.1’ saved [261978986/261978986]



In [12]:
ls -ltr

total 511692
drwxrwxr-x 2 root root      4096 Nov 21  2019 ml-25m/
-rw-r--r-- 1 root root 261978986 Dec  3  2019 ml-25m.zip.1
-rw-r--r-- 1 root root 261978986 Dec  3  2019 ml-25m.zip
drwxr-xr-x 1 root root      4096 Feb 27 14:36 sample_data/


In [13]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [14]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
replace ml-25m/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/tags.csv         
replace ml-25m/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/links.csv        
replace ml-25m/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/README.txt       
replace ml-25m/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/ratings.csv      y
y

replace ml-25m/genome-tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: ml-25m/genome-tags.csv  
replace ml-25m/genome-scores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: ml-25m/genome-scores.csv  
replace ml-25m/movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-25m/movies.csv       


In [15]:
!ls -ltr

total 511692
-rw-r--r-- 1 root root 261978986 Dec  3  2019 ml-25m.zip.1
-rw-r--r-- 1 root root 261978986 Dec  3  2019 ml-25m.zip
drwxr-xr-x 1 root root      4096 Feb 27 14:36 sample_data
drwxrwxr-x 2 root root      4096 Mar  1 15:39 ml-25m


### Start

In [16]:
cd ml-25m

/content/ml-25m


In [17]:
pwd

'/content/ml-25m'

In [18]:
!ls -ltr

total 1129584
-rw-rw-r-- 1 root root  38810332 Nov 21  2019 tags.csv
-rw-rw-r-- 1 root root 678260987 Nov 21  2019 ratings.csv
-rw-rw-r-- 1 root root   3038099 Nov 21  2019 movies.csv
-rw-rw-r-- 1 root root     18103 Nov 21  2019 genome-tags.csv
-rw-rw-r-- 1 root root 435164157 Nov 21  2019 genome-scores.csv
-rw-rw-r-- 1 root root   1368578 Nov 21  2019 links.csv
-rw-rw-r-- 1 root root     10460 Nov 21  2019 README.txt


In [19]:
from pyspark.sql import SparkSession

In [22]:
ratings_file = "/content/ml-25m/ratings.csv"
ratings_df = spark.read.csv(ratings_file, inferSchema=True, header=True)

In [23]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [24]:
# slicing
ratings_df = ratings_df.select(["userId", "movieId", "rating"])

In [25]:
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [26]:
ratings_df.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423535|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [27]:
train_df, test_df = ratings_df.randomSplit([0.8, 0.2])

In [28]:
from pyspark.ml.recommendation import ALS

In [29]:
# als parameter configuration
als = ALS(
    maxIter=5, # iteration
    regParam=0.1, # regularization
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop" # cold start 발생 시 대처 방법 (nan or drop)
)

In [30]:
model = als.fit(train_df)

In [31]:
predictions = model.transform(test_df)

In [32]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|    665|   5.0| 4.1612363|
|     1|   1217|   3.5|   4.12036|
|     1|   1260|   3.5|  4.130878|
|     1|   1653|   4.0|  3.631819|
|     1|   2012|   2.5|  2.678013|
|     1|   2068|   2.5| 4.0510526|
|     1|   2161|   3.5|  3.234428|
|     1|   2351|   4.5|  4.119405|
|     1|   5684|   2.0| 3.3134787|
|     1|   5912|   3.0| 3.7506564|
|     1|   5952|   4.0| 3.6341386|
|     1|   6539|   3.5| 3.2658343|
|     1|   7209|   4.0|  3.739603|
|     1|   7323|   3.5| 3.9935312|
|     1|   7820|   2.5| 3.9341695|
|     1|   8327|   5.0| 3.9537928|
|     1|   8360|   4.0| 3.2294827|
|     1|   8405|   3.5| 3.7929044|
|     1|  27721|   3.0| 3.8527431|
|    12|     10|   3.0| 2.9809287|
+------+-------+------+----------+
only showing top 20 rows



In [33]:
predictions.select('rating', 'prediction').describe().show()
# 실제값과 예측값 describe

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4998474|           4998474|
|   mean|3.5337971148794614|3.4241413981254314|
| stddev|1.0605669324641536|0.6431671321158869|
|    min|               0.5|        -0.8122263|
|    max|               5.0|          6.968294|
+-------+------------------+------------------+



In [34]:
from pyspark.ml.evaluation import RegressionEvaluator

# evaluator 객체 생성 
evaluator = RegressionEvaluator(metricName="rmse", labelCol='rating', predictionCol='prediction')

In [35]:
rmse = evaluator.evaluate(predictions)

In [36]:
print(rmse)

0.8099413660670133


In [37]:
# 추천 목록 3개까지 전부 보여준다
model.recommendForAllUsers(3).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{200930, 5.66427...|
|     6|[{200930, 6.07457...|
|    12|[{200930, 5.57237...|
|    13|[{151989, 6.13397...|
|    16|[{151989, 6.22031...|
|    22|[{149484, 6.33244...|
|    26|[{203086, 5.61842...|
|    27|[{203086, 5.92720...|
|    28|[{201821, 6.98940...|
|    31|[{200930, 3.86672...|
|    34|[{151989, 6.09752...|
|    44|[{151989, 7.08428...|
|    47|[{203086, 5.03574...|
|    52|[{203086, 6.01045...|
|    53|[{194334, 6.86318...|
|    65|[{199187, 6.15024...|
|    76|[{200930, 5.89751...|
|    78|[{200930, 7.02235...|
|    81|[{159305, 4.32656...|
|    85|[{181811, 5.54184...|
+------+--------------------+
only showing top 20 rows



In [38]:
model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|      1|[{3195, 5.5648766...|
|     12|[{87426, 5.522016...|
|     13|[{108346, 5.43779...|
|     22|[{87426, 5.292430...|
|     26|[{18230, 5.077716...|
|     27|[{87426, 5.543174...|
|     28|[{128167, 5.42273...|
|     31|[{87426, 5.348231...|
|     34|[{128562, 5.29205...|
|     44|[{87426, 5.308718...|
|     47|[{156252, 5.65714...|
|     52|[{115651, 4.85182...|
|     53|[{132629, 5.17100...|
|     65|[{87426, 5.453223...|
|     76|[{87426, 5.597873...|
|     78|[{87426, 4.985826...|
|     81|[{87426, 5.069320...|
|     85|[{34485, 4.962591...|
|     93|[{87426, 5.145534...|
|    101|[{105801, 5.00953...|
+-------+--------------------+
only showing top 20 rows



In [39]:
from pyspark.sql.types import IntegerType

user_list = [65, 78, 81]
users_df = spark.createDataFrame(user_list, IntegerType()).toDF('userId')

users_df.show()

+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [40]:
user_recs = model.recommendForUserSubset(users_df, 5)

In [41]:
movies_list = user_recs.collect()[0].recommendations

In [42]:
recs_df = spark.createDataFrame(movies_list)
recs_df.show()

+-------+-----------------+
|movieId|           rating|
+-------+-----------------+
| 199187|6.150240421295166|
| 126941| 6.09591817855835|
| 151989|5.981280326843262|
| 107252|5.929398059844971|
| 205277|5.802688121795654|
+-------+-----------------+



In [44]:
movies_file = "/content/ml-25m/movies.csv"
movies_df = spark.read.csv(f"file:///{movies_file}", inferSchema=True, header=True)

In [45]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [46]:
recs_df.createOrReplaceTempView("recommendations") # SQL DB
movies_df.createOrReplaceTempView("movies") # SQL DB

In [47]:
query = """
SELECT *
FROM
    movies JOIN recommendations
    ON movies.movieId = recommendations.movieId
ORDER BY
    rating desc
"""
recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+--------------------+-------+-----------------+
|movieId|               title|              genres|movieId|           rating|
+-------+--------------------+--------------------+-------+-----------------+
| 199187|       Hoaxed (2019)|  (no genres listed)| 199187|6.150240421295166|
| 126941|Joni's Promise (2...|              Comedy| 126941| 6.09591817855835|
| 151989|    The Thorn (1971)|              Comedy| 151989|5.981280326843262|
| 107252|Island at War (2004)|           Drama|War| 107252|5.929398059844971|
| 205277|   Inside Out (1991)|Comedy|Drama|Romance| 205277|5.802688121795654|
+-------+--------------------+--------------------+-------+-----------------+



In [48]:
def get_recommendations(user_id, num_recs):
    users_df = spark.createDataFrame([user_id], IntegerType()).toDF('userId') 
    user_recs_df = model.recommendForUserSubset(users_df, num_recs)
    
    recs_list = user_recs_df.collect()[0].recommendations
    recs_df = spark.createDataFrame(recs_list)
    recommended_movies = spark.sql(query)
    return recommended_movies

In [49]:
recs = get_recommendations(456, 10)

In [50]:
recs.toPandas()

,movieId,title,genres,movieId,rating
0,199187,Hoaxed (2019),(no genres listed),199187,6.150240
1,126941,Joni's Promise (2005),Comedy,126941,6.095918
2,151989,The Thorn (1971),Comedy,151989,5.981280
3,107252,Island at War (2004),Drama|War,107252,5.929398
4,205277,Inside Out (1991),Comedy|Drama|Romance,205277,5.802688


In [51]:
spark.stop()